#Problème - Session n°2

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader

class Problem1Dataset(Dataset):
    def __init__(self, num_samples=50000, group_size=1000, k=5, n=3, dim=10):
        self.num_samples = num_samples
        self.group_size = group_size
        self.k = k
        self.n = n
        self.dim = dim

        torch.manual_seed(42)

        # Generate random matrices M[k, n, dim, dim]
        self.M = torch.randn(k, n, dim, dim)

        # Generate input data
        self.x_data = torch.randn(num_samples, dim)

        # Generate L values (shared within each group)
        self.L_data = torch.arange(num_samples // group_size).repeat_interleave(group_size) % k

        # Compute target values
        self.y_data = torch.zeros(num_samples, 1)
        for i in range(num_samples):
            L_i = self.L_data[i]
            x_i = self.x_data[i]
            y_i = torch.max(self.M[L_i, 2] @ torch.relu(self.M[L_i, 1] @ torch.relu(self.M[L_i, 0] @ x_i)))
            self.y_data[i] = y_i

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx], self.L_data[idx], idx

Dans ce problème, on travaille sur un jeu de données comportant 50.000 entrées $x_i$ et des cibles $y_i$. Les entrées sont des vecteurs de taille 10 (au format torch), les cibles sont des scalaires construits à partir de cinq fonctions différentes ($f_0$, ..., $f_4$) : \

$$ \forall i, \exists k\in [0,4] \:\: \text{tel que} \: f_k(x_i) = y_i $$

Ces fonctions sont inconnues, ainsi que l'indice $k$. Par contre, on sait que le groupe des 1000 premières cibles ont été construites à partir du même indice  $k$, de même pour les mille  suivantes, et ainsi de suite.

Le but est de parvenir à rassembler les groupes de cibles qui ont été générées avec le même indice $k$ (avec la même fonction).

In [ ]:
# Example d'échantillonnage du dataset
import torch
from torch.utils.data import DataLoader

! git clone https://github.com/XXXXXXX/exam_2025_session2.git
! cp exam_2025_session2/utils/utils.py .

from utils import Problem1Dataset
dataset = Problem1Dataset()
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Iterate through one batch
for batch in dataloader:
    x_batch, y_batch, k_batch, idx_batch = batch
    print("Batch input shape:", x_batch.shape)
    print("Batch target shape:", y_batch.shape)
    print("Batch k shape:", k_batch.shape) # ground truth (non available to train the model)
    print("Batch indices shape:", idx_batch.shape) # index among the 50.000 tensors
    break  # Just to show one batch

**Consignes :**
- Entraîner l'architecture proposée dans la cellule suivante.
- Préciser le rôle de self.theta.
- Montrer que les vecteurs 2D de self.theta permettent de répondre
  au problème posé.
- Décrire le rôle du vector noise.

In [ ]:
class DeepMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepMLP, self).__init__()
        self.theta = nn.Parameter(torch.randn(50, 2))
        self.fc1 = nn.Linear(input_dim + 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, indices):
        theta_batch = self.theta[indices // 1000, :]
        noise = torch.normal(mean=torch.zeros_like(theta_batch),
                             std=torch.ones_like(theta_batch))
        x = torch.cat([x, theta_batch + noise], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x, theta_batch